

## 1. Two Sum

<https://leetcode.com/problems/two-sum/description/>

给定一个整数数列，找出其中和为特定值的那两个数。

你可以假设每个输入都只会有一种答案，同样的元素不能被重用。

示例:

```
Given nums = [2, 7, 11, 15], target = 9,

Because nums[0] + nums[1] = 2 + 7 = 9,
return [0, 1].
```

###  暴力算法

最直接的方法就是暴力算法。遍历所有的和，算法复杂度是 $O(n^2)$

In [1]:
class Solution:
    def twoSum(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: List[int]
        """
        l = len(nums)
        for i in range(l):
            for j in range(i+1, l):
                if nums[i] + nums[j] == target:
                    return [i, j]
                
Solution().twoSum([2, 7, 11, 15], 9)

[0, 1]


###  使用字典 （hash table）

观察一下 `[2, 7, 11, 15]`， 可以直接计算其解为 `[7, 2, ...]` ，存储 解和当前索引到字典中。 在往后的遍历中，先看看当前的数在不在解集合中，如果在，则直接返回之前的索引和当前的索引。如果不在，保存解和当前索引。

只需一次遍历即可，算法时间 $O(n)$

In [16]:
class Solution:
    def twoSum(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: List[int]
        """
        solution = dict()
        for i, n in enumerate(nums):
            if n in solution:
                return [solution[n], i]
            else:
                solution[target - n] = i

%time Solution().twoSum([2, 7, 11, 15], 9)

CPU times: user 16 µs, sys: 1 µs, total: 17 µs
Wall time: 24.1 µs


[0, 1]

## 2. Add Two Numbers

两个链表分别表示两个数字，从低位指向高位的方向。求和

```
Input: (2 -> 4 -> 3) + (5 -> 6 -> 4)
Output: 7 -> 0 -> 8
Explanation: 342 + 465 = 807.
```

咋看很简单啊。 注意一些细节。 这里返回一个新的链表

借用了 python 的 heapq.merge程序的思想， 用两个循环而不是一个循环。最后一个循环处理剩余的部分。

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

class Solution:
    def addTwoNumbers(self, l1, l2):
        """
        :type l1: ListNode
        :type l2: ListNode
        :rtype: ListNode
        """
        
        x1 = l1
        x2 = l2
        carry = 0
        head = prev = ListNode(0)
        # loop1
        while (x1 and x2):
            val = x1.val + x2.val + carry
            prev.next = ListNode(val % 10)
            carry = val // 10
            prev = prev.next
            x1 = x1.next
            x2 = x2.next
            
        rx = x1 if x2 is None else x2
        
        # loop2
        while rx:
            val = rx.val + carry
            prev.next = ListNode(val % 10)
            carry = val // 10
            prev = prev.next
            rx = rx.next
            
        if carry:
            prev.next = ListNode(carry)
        
        return head.next
            

In [25]:
def create_list_node(num):
    head = prev = ListNode(0)
    while num:
        prev.next = ListNode(num % 10)
        num = num // 10
        prev = prev.next
    return head.next

def parse_list_node(ln):
    num = 0
    base = 1
    while ln:
        num += ln.val * base
        base = base * 10
        ln = ln.next
        
    return num 

def print_list_node(ln):
    l = list()
    while ln:
        l.append(ln.val)
        ln = ln.next
    print(l)
    
print_list_node(create_list_node(100))

print(parse_list_node(create_list_node(1024)))
assert parse_list_node(create_list_node(100)) == 100

def test(a, b):
    # create
    l1 = create_list_node(a)
    l2 = create_list_node(b)
    s = Solution().addTwoNumbers(l1, l2)
#     print_list_node(s)
    assert parse_list_node(s) == (a + b)
    
test(123, 3452323)

import random

for _ in range(100):
    a = random.randint(0, 100000000)
    b = random.randint(0, 100000000)
    test(a, b)
    

[0, 0, 1]
1024


## 3. Longest Substring Without Repeating Characters

Given a string, find the length of the longest substring without repeating characters.

Examples:

Given "abcabcbb", the answer is "abc", which the length is 3.

Given "bbbbb", the answer is "b", with the length of 1.

Given "pwwkew", the answer is "wke", with the length of 3. Note that the answer must be a substring, "pwke" is a subsequence and not a substring.


返回没有重复字符的最长子串的长度。

思路很直接啊： 用头尾指针夹住无重复字符的子串。
1. 需要一前一后两个指针：尾指针遍历字符串， 当遇到重复字符时，更新头指针位置。必须总能确保：头尾指针之间没有重复字符。
2. 注意上面有个trick。 一般使用字典保存字符位置。只需要保证头尾指针之间没有重复字符即可，所以重复的指针还有一个必要条件是 `i <= used[c]`。
3. 遍历时计算长度，更新最大长度。
    
注意一些细节：

```
i 0--1--4-
  |  |  |
  abcadeab
  ||||||||
j 01234567
l 123-45-4
```

注意下面这个例子，必须同时满足 `i <= used[c]` 才更新头指针位置。如果 `i > used[c]`，`c` 已经可以不用考虑了。
只考虑 `i` 和 `j` 之间是否有重复的字符。
```
i 0-2----         
  | |   
  tmmzuxt   
  |||||||
j 0123456
l 12-2345
```

In [62]:
class Solution:
    def lengthOfLongestSubstring(self, s):
        """
        :type s: str
        :rtype: int
        """
        used = {}
        ml = i = 0
        
        for j, c in enumerate(s):
            if c in used and i <= used[c]:
                i = used[c] + 1
            else:
                ml = max(ml, j - i + 1)

            used[c] = j
#             print(i,j,ml)
        return ml
    
# print(Solution().lengthOfLongestSubstring('abcabcbb'))  # 3
# print(Solution().lengthOfLongestSubstring('bbbbbb'))  # 1
# print(Solution().lengthOfLongestSubstring('pwwkew'))  # 3
print(Solution().lengthOfLongestSubstring('abcadeab'))  # 5
# print(Solution().lengthOfLongestSubstring(''))  # 0
# print(Solution().lengthOfLongestSubstring('c'))  # 1
# print(Solution().lengthOfLongestSubstring('abcdefg'))  # 7
# print(Solution().lengthOfLongestSubstring("tmmzuxt"))  # 5


5


## 4. Median of Two Sorted Arrays

There are two sorted arrays nums1 and nums2 of size m and n respectively.

Find the median of the two sorted arrays. The overall run time complexity should be O(log (m+n)).

Example 1:
```
nums1 = [1, 3]
nums2 = [2]

The median is 2.0
```
Example 2:
```
nums1 = [1, 2]
nums2 = [3, 4]

The median is (2 + 3)/2 = 2.5
```

解题思路，很巧妙，参考讨论区的来着：

什么是中位数，就是排序数组中间的两个数（或一个数）的平均数。

那么两个数组的中位数怎么找。 注意把两个有序数组这样排放（总是假设 A长度 $m$ 不大于 B长度 $n$）：
```
A:      [........]
B:  [.................]
```
在中心位置（旋转对称轴中心？）切一刀，例如：

```
A:      [....|....]
B:  [........|.........]
```
或者

```
A:      [\........]
          \
           \
            \
             \
              \
               \
                \ 
                 \
B:  [.............\....]   ^_^
```
保证左侧的和右侧的数量相等（或者左边的比右边的多1个，如果总数是奇数的话）。当左侧数总是不大于右侧数的时候，其实就是找到了中位数的位置。

上面的切割位置 $i \in [0, m]$， 下面的切割位置 $j \in [(n - m)/2, (n + m)/2]$。

如果总数是偶数，则 $i,j$ 满足： $i + j = (n + m) /2 $。

如果总数是奇数，则 $i,j$ 满足： $i + j = (n + m + 1) / 2 $。 统一写就是 $i + j = (n + m + 1 ) / 2$， 偶数的0.5会舍去。

所以问题本质上就是个二分查找题，查找范围 $i \in [0, m]$， 下边数组的切割位置 $j = (n + m + 1) / 2 - i$

其他细节：

1. 对于切割位置 $i,j$， 如果 $A[i-1] > B[j]$ 的话，同时说明 $A[i] \ge A[i-1] > B[j] \ge B[j-1]$， 说明在 $i$ 切割位置，上面的数普遍比下面的数大，$i$ 应该偏小一点。收缩查找范围到 $[imin, i-1]$
2. 如果 $B[j-1] > A[i]$ 的话， 同时说明 $B[j] \ge B[j-1] > A[i] \ge A[i-1]$，说明 $i$ 的切割位置，上面的数普遍比下面的数小，$i$ 应该偏大一些。收缩查找范围到 $[i+1, imax]$
3. 如果总数是奇数的话，返回左侧数的最大值即可； 如果总数是偶数的话，需要求个平均值。同时还要注意边界条件。


思路很清晰，但是边界条件以及细节都很麻烦。 自己写不知道写到何时去。。。。。。。。

In [65]:
class Solution:
    def findMedianSortedArrays(self, nums1, nums2):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :rtype: float
        """ 
        m, n = len(nums1), len(nums2)
        if m > n:
            nums1, nums2, m, n = nums2, nums1, n, m
        
        A, B = nums1, nums2
        imin, imax, half_len = 0, m, (m + n + 1) // 2
        while imin <= imax:
            i = (imin + imax) // 2
            j = half_len - i
            if i < m and B[j-1] > A[i]:
                # i is too small, must increase it
                imin = i + 1
            elif i > 0 and A[i-1] > B[j]:
                # i is too big, must decrease it
                imax = i - 1
            else:
                # i is perfect

                if i == 0: max_of_left = B[j-1]
                elif j == 0: max_of_left = A[i-1]
                else: max_of_left = max(A[i-1], B[j-1])

                if (m + n) % 2 == 1:
                    return max_of_left

                if i == m: min_of_right = B[j]
                elif j == n: min_of_right = A[i]
                else: min_of_right = min(A[i], B[j])

                return (max_of_left + min_of_right) / 2.0
            
Solution().findMedianSortedArrays([1, 3], [2])

2

## 5. Longest Palindromic Substring

回文字符串。

题意很简单： 找出一个字符串中最长的回文字符串。

例如： `ababa` 就是一个回文字串。

用到动态规划思想。简单回顾一下动态规划思想，动态规划一般用在求最优解的问题中。 动态规划包括以下4个步骤：

1. 描述最优解的结构， 并且可以分解成子问题的最优解的选择。
2. 递归定义最优解的值，即子问题组合
3. 按自底向上的方式计算出最优解的值
4. 由计算出的结果构造一个最优解

说的比较抽象，关键点在于 最优解的结构可以分解成子问题的最优解的选择。

### 一般解法

回文字符串没这么麻烦，其实很直观： 遍历每个字符，以每个字符为中心向两边扩展搜索，直到不满足回文结构为止。 这种方法的时间复杂度是 $O(n^2)$， 空间复杂度是 $O(1)$，其实可以就地搜索。

经常就是边界条件的trick：

1. 关于 `expand`， 
    1. 如果 `i==j`，就是从字符展开，得到的长度是奇数； 如果 `j = i + 1`，就是从两个字符展开，得到的长度是偶数。
    2. 如果 `i==j`，那么一定有 `s[i] == s[j]`， 直接计算 `j - i` 得到的是偶数， 需要减1
    3. 如果 `j = i + 1`，如果 `s[i] == s[j]`， 计算 `j - i` 得到 3， 需要减1。
2. 关于计算回文子串的起止位置：
    1. 如果 l  是奇数， 表示从1个字符展开， `j = i - l // 2 , k = i + l // 2`
    2. 如果 l  是偶数， 表示从2个字符展开， `j = i - l // 2 - 1, k = i + l // 2`
    3. 可以看到 k 的表达式都一样。 j 的表达式需要调和一下， 可以变成 `j = i - (l - 1) // 2`， 符合奇偶情况。

In [ ]:
class Solution:
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        k = j = 0
        for i in range(len(s)):
            l1 = self.expand(s, i, i)
            l2 = self.expand(s, i, i+1)
            l = max(l1, l2)
            if l > k - j:
                j = i - (l - 1) // 2
                k = i + l // 2
        return s[j:k+1]
            
    def expand(self, s, i, j):
        while (i >=0 and j < len(s) and s[i] == s[j]):
            i -= 1
            j += 1
        return j - i - 1
    
Solution().longestPalindrome('abaabadddd')

### Manacher’s Algorithm

核心思想： 利用 回文子串的特点： 对回文的另一侧子串省略扩展搜索

具体的解释见： <https://articles.leetcode.com/longest-palindromic-substring-part-ii/>

一些细节：

1. 在头和尾加上 '^$' 主要是为了避免ns的index错误。
2. 插入 '#‘ 可以巧妙优雅的简化问题。详见上面的链接。
3. r表示已知的回文子串的最大右边界（在这个范围内可以采取一些镜像策略）， c表示这个回文子串的中心位置。
4. `p = min(r - i, P[i_mirror]) if r > i else 0` 是关键的trick。
    1. 拆开解释： 如果 `r <= i`，表示 i 超出了最大右边界，这时候就老老实实从这里扩展，找 `p`
    2. 如果 `r > i`，比较 `r - i` 即i到右边界的剩余部分，和镜像中心回文子串长度 `P[i_mirror]`
        1. 如果 剩余部分多，完全可以镜像过来。所以 `p = P[i_mirror]`
        2. 如果 剩余部分少，右边界以内的肯定是回文的，但是右边界以外的就需要扩展搜索。（但是逻辑上，没必要继续扩展搜索。因为如果右边界以外也符合回文条件，那么镜像范围是增大的，之前的最大右边界条件不成立）
        3. 所以用 `min(r - i, P[i_mirror])`
5. 如果从 i 开始的扩展搜索的范围超过了右边界，那么就定义新的中心和边界位置。
5. 最后用到 `argmax` 找到最长回文子串的中心位置。
6. 用替换比用计算原字符串的位置，不仅省事，还省时间。。。。不纠结

原始算法的复杂度是 $O(n^2)$， 这个算法的时间复杂度可以达到 $O(n)$，为什么，可以感觉到： 到达右边界的搜索过程是线性的。在到达右边界后，在右边界之内的就**不用重复搜索，只需要线性时间的计算而已**。 虽然是两个循环嵌套，但好比你追我赶，所以就是线性的。

In [105]:
class Solution:
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        ns = '^#' + '#'.join(s) + '#$'
        P  = [0]
        c = r = 0
        for i in range(1, len(ns)-1):
            i_mirror = 2 * c - i
            
            p = min(r - i, P[i_mirror]) if r > i else 0
            while (ns[i + p + 1] == ns[i - p - 1]):
                p += 1
            
            if i + p > r:
                c = i
                r = i + p
            
            P.append(p)
            
        center, length = max(enumerate(P), key=lambda x:x[1]) 
        return ns[center - length: center + length].replace('#', '')

Solution().longestPalindrome('abaabadddd')

'abaaba'

## 682. Baseball Game


https://leetcode.com/problems/baseball-game/description/

很简单的题目，baseball计分规则：

1. 数字，就是分数
2. 'D'， 就是把前一轮的有效分数翻倍，作为本轮分数
3. '+'， 就是把前两轮有效分数相加，作为本轮分数
4. 'C'， 是一个操作，把前一轮分数定为无效。

给出一列计分，求最后的分数总和。

例如 :

`["5","2","C","D","+"]` ，最后得分是 30分。

用栈就可以解决。 算法时间复杂度 $O(n)$， 空间复杂度 $O(n)$。

In [67]:
class Solution:
    def calPoints(self, ops):
        """
        :type ops: List[str]
        :rtype: int
        """
        points = []
        for op in ops:
            if op == 'C':
                points.pop()
            elif op == 'D':
                points.append(points[-1] * 2)
            elif op == '+':
                points.append(points[-1] + points[-2])
            else:
                points.append(int(op))
        return sum(points)
        
    
Solution().calPoints(["5","2","C","D","+"])

30

## 6. ZigZag Conversion

中等难度的问题？？

比如将  'ABCDEFGHIJKLMNOP' 变成 4行 zigzag型的：

```
A  G  M
B FH LN
CE IK O
D  J  P
```
然后按行输出 'AGMBFHLNCEIKODJP'

我的心路历程：

1. matrix？，有点太大了，费空间
2. 计算索引？，似乎有点用，但是太麻烦了
3. 看了讨论，有一个很简单很直接的做法：用列表。。。。。。

简单点说，就是有多少行，用多少个字符串。 每行的字符串按顺序缀上字符就可以了。。

一个技巧是提前准备好行的索引：

比如4行的索引是 `[0, 1, 2, 3, 2, 1]` 重复，可以用 `list(range(numRows-1)) + list(range(numRows-1, 0, -1))`

In [126]:
## 80多ms

class Solution:
    def convert(self, s, numRows):
        """
        :type s: str
        :type numRows: int
        :rtype: str
        """
        if numRows == 1 or numRows >= len(s):
            return s
        
        rows = [''] * numRows
        index = list(range(numRows-1)) + list(range(numRows-1, 0, -1))
        n = len(index)
        for i, c in enumerate(s):
            rows[index[i%n]] += c
        return ''.join(rows)

print(Solution().convert("", 1))
print(Solution().convert("ABCDEFGHIJKLMNOP", 4))


AGMBFHLNCEIKODJP


In [127]:
# 108ms

class Solution:
    def convert(self, s, numRows):
        """
        :type s: str
        :type numRows: int
        :rtype: str
        """
        if numRows == 1 or numRows >= len(s):
            return s
        
        rows = []
        for i in range(numRows):
            rows.append(list())
        index = list(range(numRows-1)) + list(range(numRows-1, 0, -1))
        n = len(index)
        for i, c in enumerate(s):
            rows[index[i%n]].append(c)
        return ''.join((''.join(row) for row in rows))

print(Solution().convert("", 1))
print(Solution().convert("ABCDEFGHIJKLMNOP", 4))


AGMBFHLNCEIKODJP


## 7. Reverse Integer



In [136]:
class Solution:
    def reverse(self, x):
        """
        :type x: int
        :rtype: int
        """
        sign = ''
        if x < 0:
            sign = '-'
            x = -x
        s = str(x)
        rx = int(sign + s[::-1])
        if -2147483649 < rx < 2147483648:
            return rx
        else:
            return 0

print(Solution().reverse(1234))
print(Solution().reverse(-1234))
print(Solution().reverse(2222222222222222222))
# print(Solution().reverse(1234))
        
        

4321
-4321
0


In [139]:

## 改进版，抄的，速度不变，更好看一些而已

class Solution:
    def reverse(self, x):
        """
        :type x: int
        :rtype: int
        """
        if x < 0:
            y = -1 * int(str(-x)[::-1])
        else:
            y = int(str(x)[::-1])  
            
        if y > 2**31 or y < -2**31:
            y = 0
        return y

print(Solution().reverse(1234))
print(Solution().reverse(-1234))
print(Solution().reverse(2222222222222222222))
# print(Solution().reverse(1234))

4321
-4321
0


In [ ]:
a = list(range(100000000))
%timeit len(a)
del a

## 8. String to Integer (atoi)

这个没啥好说的。挑战是 要应对所有的输入情况

Hint: Carefully consider all possible input cases.


1. 去掉首部空白字符：有效字符： +-0123456789， 需满足正则表达式： `[+-]?\d+` 
2. 去掉末尾的其他无效字符。
3. 如果首部遇到无效字符，则返回0。  如果转换的数字超过了有效范围，返回上下限。

按照顺序写代码就可以了。很简单。


In [22]:
class Solution:
    def myAtoi(self, s):
        """
        :type str: str
        :rtype: int
        """
        if len(s) == 0:
            return 0
        
        INT_MAX = 2147483647
        INT_MIN = -2147483648
        
        i = 0
        while (s[i] == ' '):
            i += 1
        
        neg = False
        if s[i] == '+':
            i += 1
        elif s[i] == '-':
            i += 1
            neg = True
            
        num = 0
        n_digits = 0
        for j in range(i, len(s)):
            d = ord(s[j]) - 48  # ord('0')
            if d < 0 or d > 9:
                break
            num = num * 10 + d
            n_digits += 1
            if n_digits > 10:  # avoid too long digits
                break
        
        if neg:
            num = -num
        if num > INT_MAX:
            return INT_MAX
        elif num < INT_MIN:
            return INT_MIN
        else:
            return num

print(Solution().myAtoi('123'))
print(Solution().myAtoi('  sdlfjklsd'))
print(Solution().myAtoi('  +12345'))
print(Solution().myAtoi(' - 1234'))
print(Solution().myAtoi('123'))
print(Solution().myAtoi('-'))


123
0
12345
0
123
0


##  9. Palindrome Number

Determine whether an integer is a palindrome. Do this without extra space.

注意两点：

1. 负数不算，必须是整数
2. 不能用额外的 $O(n)$ 空间。

这个题目的意义？？ 话说回来，感觉好多算法题的意义都不是很大。


注意这里有个陷阱是： 如果 在整型范围类， 翻转数字可能溢出。 （在python中没这个问题）。 有个解决思路是： 只翻转一半的数字。

In [35]:
class Solution:
    def isPalindrome(self, x):
        """
        :type x: int
        :rtype: bool
        """
        if 0 <= x < 10:
            return True
        if x % 10 == 0:
            return False
        
        y = 0
        while (x > y):
            y = y * 10 + x % 10
            x = x // 10
            if y == x or y == x // 10:
                return True
            
        return False

print(Solution().isPalindrome(123454321))
print(Solution().isPalindrome(12))
print(Solution().isPalindrome(10))
print(Solution().isPalindrome(100))
print(Solution().isPalindrome(1000))
print(Solution().isPalindrome(100000))

True
False
False
False
False
False


下面是 无视题意的做法。。

In [37]:
class Solution:
    def isPalindrome(self, x):
        """
        :type x: int
        :rtype: bool
        """
        s = str(x)
        return s == s[::-1]

print(Solution().isPalindrome(123454321))
print(Solution().isPalindrome(12))
print(Solution().isPalindrome(10))
print(Solution().isPalindrome(100))
print(Solution().isPalindrome(1000))
print(Solution().isPalindrome(100000))

True
False
False
False
False
False


## 10. Regular Expression Matching

这题比较难，就是实现 正则表达式的 `.` 和`*`

- '.' 表示匹配任意某字符
- '*’ 表示匹配0到多个字符


一开始我的想法比较单纯。。用两个指针

1. `p[j] == '.' and p[j+1] == '*'`  ，找完 `i`， `j += 2`
2. `p[j] == '.' and p[j+1] != '*'`  ，`i += 1`, `j += 1`
3. `p[j] == s[i] and p[j+1] == '*'` ，找完 `i`， `j += 2`
4. `p[j] == s[i] and p[j+1] != '*'` ，`i += 1`, `j += 1`
5. `p[j] != s[i] and p[j+1] == '*'`， `j += 2`
6. `p[j] != s[i] and p[j+1] != '*'`， 返回False

最后，如果 i 和 j 都没到末尾，那么也返回False。 否则返回True。

后来发现事情没那么简单。。

这种方式对 匹配不了 'aaa' 和 `a*a`

In [46]:
class Solution:
    def isMatch(self, s, p):
        """
        :type s: str
        :type p: str
        :rtype: bool
        """
        i = j = 0
        ls = len(s)
        lp = len(p)
        while i < ls and j < lp:
            if j + 1 < lp and p[j+1] == '*':
                if p[j] == '.':
                    i = ls   # 耗尽                    
                elif p[j] == s[i]:
                    while i < ls and s[i] == p[j]:
                        i += 1
                j += 2
            else:
                if p[j] == '.' or p[j] == s[i]:
                    i += 1
                    j += 1
                else:
                    return False
        
        if i < ls or j < lp:
            return False
        
        return True
    
print(Solution().isMatch('aa', 'a'))
print(Solution().isMatch("aa","aa"))
print(Solution().isMatch("aaa","aa"))
print(Solution().isMatch("aa", "a*"))
print(Solution().isMatch("aa", ".*"))
print(Solution().isMatch("ab", ".*"))
print(Solution().isMatch("aab", "c*a*b"))
print(Solution().isMatch("aaa", "a*a"))

False
True
False
True
True
True
True
False


 正确的算法是 动态规划。找到最优子结构。
 
 有最优子结构： `dp(i, j) = isMatch( s[i:], p[j:])` 。 
 
这样分解和选择：注意 `.*` 或者 `a*` 可以匹配空字符串。

- `s[i] == p[j] or p[j] == '.', p[j+1] == '*'`, `dp(i, j) = dp(i + 1, j)`， 因为`.*`可以匹配空字符串
- `s[i] != p[j], p[j+1] == '*'`, `dp(i, j) = dp(i + 1, j) and dp(i, j+2)`， 因为当前位置字符不匹配，则`.*` 一定匹配的是空字符串， 查看 i 与 j+2 的匹配情况。

下面如果 `p[j+1] != *`：

- `s[i] == p[j] or p[j] == '.'`, `dp(i, j) = dp(i + 1, j + 1)`
- `s[i] != p[j]`, `dp(i, j) = False`

 注意下面，先循环 j， 再循环 i。 总是解决了子问题。
 
 
 算法时间复杂度 $O(TP)$ ，空间复杂度 $O(TP)$

In [47]:
class Solution(object):
    def isMatch(self, text, pattern):
        dp = [[False] * (len(pattern) + 1) for _ in range(len(text) + 1)]

        dp[-1][-1] = True
        for i in range(len(text), -1, -1):
            for j in range(len(pattern) - 1, -1, -1):
                first_match = i < len(text) and pattern[j] in {text[i], '.'}
                if j+1 < len(pattern) and pattern[j+1] == '*':
                    dp[i][j] = dp[i][j+2] or first_match and dp[i+1][j]
                else:
                    dp[i][j] = first_match and dp[i+1][j+1]

        return dp[0][0]
    
print(Solution().isMatch('aa', 'a'))
print(Solution().isMatch("aa","aa"))
print(Solution().isMatch("aaa","aa"))
print(Solution().isMatch("aa", "a*"))
print(Solution().isMatch("aa", ".*"))
print(Solution().isMatch("ab", ".*"))
print(Solution().isMatch("aab", "c*a*b"))
print(Solution().isMatch("aaa", "a*a"))

False
True
False
True
True
True
True
True


下面自己写的，规则还是很复杂的，没有DP方法优雅，并且还是错的

 自底向上就是从 最后的位置开始算起： 
 
 如果 `s[i:]` 和 `p[j:]` 匹配， 
 - 如果 `p[j-1] == '*' and p[j-2] == '.'`， `s[0:]` 和`p[j-2:]` 匹配。
 - 如果 `p[j-1] == '*' and p[j-2] == s[i-1]`, `s[i-1:]` 和`p[j-2:]` 匹配
 - 如果 `p[j-1] == '.'`， `s[i-1:]` 和 `p[j-1:]` 匹配
 - 如果 `p[j-1] == s[i-1]`， 则`s[i-1:]` 和 `p[j-1:]`匹配。
 
 最后的结果是 `s[0:]` 和 `p[0:]` 是否匹配。 为了方便，将结果保存在一个 `len(s) X len(p)` 的矩阵里。
 
 感觉又不用？？
 

In [59]:
class Solution(object):
    def isMatch(self, s, p):
        i, j = len(s), len(p)
        while (i > 0 and j > 0):
            if j > 1 and p[j-1] == '*':
                if p[j-2] == '.':
                    i = 0
                elif p[j-2] == s[i-1]:
                    while (s[i-1] == p[j-2] and i > 0):
                        i -= 1
                j -= 2
            elif p[j-1] in {s[i-1], '.'}:
                i -= 1
                j -= 1
            else:
                return False
            print(s[i:], p[j:])
                
        if i == 0:
            while (j > 1 and p[j - 1] == '*'):
                j -= 2
                    
        if i > 0 or j > 0:
            return False
        
        return True
    

# print(Solution().isMatch('aa', 'a'))
# print(Solution().isMatch("aa","aa"))
# print(Solution().isMatch("aaa","aa"))
# print(Solution().isMatch("aa", "a*"))
# print(Solution().isMatch("aa", ".*"))
# print(Solution().isMatch("ab", ".*"))
# print(Solution().isMatch("aab", "c*a*b"))
# print(Solution().isMatch("aaa", "a*a"))   
# print(Solution().isMatch("ab", ".*c"))   
print(Solution().isMatch("aaa", "ab*a*c*a"))

a a
a c*a
aaa a*c*a
False
